# 阶段一: 候选短语的自动化发现 (The Discovery Engine)

**总目标:** 从约17万条中国新闻中，通过三种互补的算法（Gensim Phrases, spaCy NER, spaCy Noun Chunks），大规模挖掘潜在的、有意义的多词短语，为后续的人工审核环节提供一个高质量的“候选池”。

---
### 步骤 1.0: 环境设置与库导入

In [ ]:
# 导入必要的库
import pandas as pd
import spacy
import time
import re
from gensim.models.phrases import Phrases
from collections import Counter
from tqdm.auto import tqdm
import pickle  # 用于保存中间结果
import multiprocessing as mp

# --- 配置区 ---

ALIYUN_OSS_PATH = ''  #ALIYUN_OSS_PATH = '/mnt/data/scripts/'

# 输入文件: 筛选并去重后的中国相关文章数据 (CSV格式)
INPUT_CHINA_DATA_PATH = ALIYUN_OSS_PATH + "../data_processed/final_china_news.csv"
# 中间产出文件: 保存分词结果，避免重复运行耗时步骤
TOKEN_LISTS_PATH = ALIYUN_OSS_PATH + "../data_processed/intermediate_raw_token_lists.pkl"
# 最终产出文件: 保存本阶段发现的所有候选短语，以便下一阶段整合
CANDIDATES_GENSIM_PATH = ALIYUN_OSS_PATH + "../data_processed/candidates_gensim.pkl"
CANDIDATES_NER_PATH = ALIYUN_OSS_PATH + "../data_processed/candidates_ner.pkl"
CANDIDATES_NOUN_CHUNKS_PATH = ALIYUN_OSS_PATH + "../data_processed/candidates_noun_chunks.pkl"

print("--- 环境准备 ---")
print(f"输入文件路径: {INPUT_CHINA_DATA_PATH}")

# --- 加载spaCy模型 ---
# 这里我们需要完整的模型来进行NER和语法分析
print("\n正在加载spaCy模型 'en_core_web_lg' (完整模式)...")
start_time = time.time()
try:
    nlp = spacy.load("en_core_web_lg", disable=["lemmatizer"])
    print(f"spaCy模型加载成功！耗时: {time.time() - start_time:.2f} 秒")
except OSError:
    print("错误: spaCy模型 'en_core_web_lg' 未安装。")
    print("请在你的终端或命令行中运行: python -m spacy download en_core_web_lg")


### 步骤 1.1: 前置任务 - 文本预分词

**逻辑与目的:**
为了让后续的`Gensim Phrases`模型能够高效运行，我们需要先将长文本切分成单词列表。此处的处理非常轻量，目的是保留尽可能多的原始短语结构，包括大小写和特殊字符组合。

**动作:**
1. 加载数据。
2. 创建分词器。
3. 批量分词，只进行最基础的过滤（非标点、非空格），保留原始文本。
4. 生成并保存内存对象 `raw_token_lists`。

In [ ]:
# --- 步骤 1.1, 1.3, 1.4: 一体化并行处理（内存优化版）---

# 1. 加载数据 (修正您指出的缺失问题)
print("--- 步骤 1: 数据加载与准备 ---")
print(f"正在从 {INPUT_CHINA_DATA_PATH} 读取数据...")
start_time_load = time.time()
try:
    df = pd.read_csv(INPUT_CHINA_DATA_PATH, dtype={'DATE': str, 'CONTENT': str})
    df.dropna(subset=['CONTENT'], inplace=True)
    print(f"数据加载完成！耗时: {time.time() - start_time_load:.2f} 秒。 数据形状: {df.shape}")
except FileNotFoundError:
    print(f"❌ 错误: 输入文件不存在: {INPUT_CHINA_DATA_PATH}")
    df = pd.DataFrame() # 创建空DataFrame以避免后续代码出错

if not df.empty:
    print("\n--- 步骤 2: 开始一体化并行处理：预分词、NER、名词短语提取 ---")
    start_time_process = time.time()

    # 初始化所有结果容器
    # 注意：这里我们仍然一次性将结果保存在内存中。
    # 如果内存非常紧张，后续可以修改为流式写入文件。
    # 但对于17万篇文章，现代PC通常可以承受这些结果对象的内存占用。
    raw_token_lists = []
    ner_candidates = Counter()
    noun_chunk_candidates = Counter()

    # 定义常量和智能配置进程数 (采纳您的建议)
    import psutil
    cpu_cores = psutil.cpu_count(logical=False) # 使用物理核心数可能更稳定
    # 根据可用内存和CPU核心数动态调整，这里设置一个经验值：每进程至少需要2GB空闲内存
    max_proc_by_mem = psutil.virtual_memory().available // (2 * 1024**3)
    n_processes = min(cpu_cores - 1 if cpu_cores > 1 else 1, int(max_proc_by_mem), 8) # 上限设为8，避免过度消耗
    if n_processes < 1: n_processes = 1

    print(f"系统CPU物理核心: {cpu_cores}, 可用内存: {psutil.virtual_memory().available / 1024**3:.2f} GB")
    print(f"将使用 {n_processes} 个进程进行一体化并行处理。")

    TARGET_ENTITY_LABELS = {'ORG', 'PERSON', 'GPE', 'NORP', 'FAC', 'LOC', 'PRODUCT', 'EVENT'}
    determiners = {'the', 'a', 'an', 'my', 'your', 'his', 'her', 'its', 'our', 'their'}

    # --- 只进行一次 nlp.pipe 循环，并启用并行处理 ---
    # batch_size 可以适当调大，例如 500-1000，以减少进程通信开销
    with nlp.select_pipes(enable=["tok2vec", "tagger", "parser", "ner"]): # 明确启用需要的组件
        for doc in tqdm(nlp.pipe(df['CONTENT'], batch_size=500, n_process=n_processes), total=len(df), desc="一体化处理中"):

            # === 任务1: 预分词 (来自步骤 1.1) ===
            tokens = [token.text for token in doc if not token.is_punct and not token.is_space]
            raw_token_lists.append(tokens)

            # === 任务2: NER提取 (来自步骤 1.3) ===
            for ent in doc.ents:
                if ent.label_ in TARGET_ENTITY_LABELS:
                    entity_text = ent.text.strip()
                    if ' ' in entity_text:
                        ner_candidates[entity_text.lower()] += 1

            # === 任务3: 名词短语提取 (来自步骤 1.4) ===
            for chunk in doc.noun_chunks:
                phrase_text = chunk.text.lower()
                words = phrase_text.split()
                if words:
                    if words[0] in determiners: words = words[1:]
                    if words and words[-1] in determiners: words = words[:-1]
                cleaned_phrase = " ".join(words).strip()
                if cleaned_phrase and ' ' in cleaned_phrase:
                    noun_chunk_candidates[cleaned_phrase] += 1

    total_time = time.time() - start_time_process
    print(f"\n一体化处理完成！总耗时: {total_time/60:.2f} 分钟。")

    # --- 步骤 3: 保存所有中间结果 ---
    print("\n--- 步骤 3: 保存所有处理结果 ---")

    print(f"正在保存预分词结果 ({len(raw_token_lists)}篇文章) 到 {TOKEN_LISTS_PATH}...")
    with open(TOKEN_LISTS_PATH, 'wb') as f:
        pickle.dump(raw_token_lists, f)
    print("保存成功！")
    # 及时清理内存 (采纳您的建议)
    del raw_token_lists

    print(f"\n正在保存NER候选结果 ({len(ner_candidates)}个) 到 {CANDIDATES_NER_PATH}...")
    with open(CANDIDATES_NER_PATH, 'wb') as f:
        pickle.dump(ner_candidates, f)
    print("保存成功！")
    del ner_candidates

    print(f"\n正在保存名词短语候选结果 ({len(noun_chunk_candidates)}个) 到 {CANDIDATES_NOUN_CHUNKS_PATH}...")
    with open(CANDIDATES_NOUN_CHUNKS_PATH, 'wb') as f:
        pickle.dump(noun_chunk_candidates, f)
    print("保存成功！")
    del noun_chunk_candidates

else:
    print("DataFrame为空，跳过处理步骤。")

### 步骤 1.2: )？？？？？候选发现策略一 - 统计共现 (Gensim `Phrases`

**逻辑与目的:**
利用统计学原理，找出那些“粘合度”很高的、频繁在一起出现的单词组合。此方法对发现**专业术语**和高频实体特别有效。

**动作:**
1.  标准化输入：创建`raw_token_lists`的小写副本。
2.  训练`Phrases`模型。
3.  导出候选短语及其得分。

In [ ]:
# --- 步骤 1.2: Gensim Phrases 发现 ---
import pickle
from gensim.models.phrases import Phrases
import os

# 确保输入文件存在
if not os.path.exists(TOKEN_LISTS_PATH):
    print(f"❌ 错误: Gensim的输入文件 {TOKEN_LISTS_PATH} 不存在。请先运行上一步骤。")
else:
    print(f"正在从 {TOKEN_LISTS_PATH} 加载预分词结果...")
    with open(TOKEN_LISTS_PATH, 'rb') as f:
        # 对于Gensim，我们需要一次性加载所有token列表来训练模型
        # 如果内存极度紧张，这里可以考虑分块训练，但会损失一些精度
        raw_token_lists = pickle.load(f)
    print("加载完成。")

    # 1. 标准化输入
    print("正在为Gensim Phrases准备小写化的token列表...")
    documents_for_gensim = [[token.lower() for token in doc] for doc in tqdm(raw_token_lists, desc="转小写")]
    del raw_token_lists # 及时清理

    # 2. 训练模型
    print("\n开始训练Phrases模型...")
    start_time = time.time()
    phrases_model = Phrases(documents_for_gensim, min_count=20, threshold=10.0, delimiter=b'_')
    train_time = time.time() - start_time
    print(f"Phrases模型训练完成！耗时: {train_time:.2f} 秒。")

    # 3. 导出候选
    print("\n正在导出Gensim发现的候选短语...")
    gensim_candidates_raw = phrases_model.export_phrases(documents_for_gensim)
    gensim_candidates = {phrase.decode('utf-8').replace('_', ' '): score for phrase, score in gensim_candidates_raw}
    print(f"Gensim Phrases发现了 {len(gensim_candidates)} 个候选短语。")

    # 4. 保存结果
    print(f"\n正在将Gensim候选结果保存到 {CANDIDATES_GENSIM_PATH}...")
    with open(CANDIDATES_GENSIM_PATH, 'wb') as f:
        pickle.dump(gensim_candidates, f)
    print("保存成功！")